In [1]:
# load csv  as panda df and sort by column
import pandas as pd
from helper import *

def load_csv(file):
    df = pd.read_csv(file)
    df = df.sort_values(by=["acc_version", "hardware", "model", "thread"])
    df = df.sort_values(by=["thread", "model", "acc_version", "hardware"])
    df.fillna(0, inplace=True)
    valid_loc = df.columns.get_loc("valid")
    for i in range(valid_loc, len(df.columns)):
        df.iloc[:, i] = df.iloc[:, i].astype(int)
    df = df.drop(df.columns[[0]], axis=1)
    df = df.drop(columns=["del", "del_version"])
    return df


def combine(df, df2):
    df2 = df2[df.columns]
    df = pd.concat([df, df2], ignore_index=True)
    return df


def merge_rows(df,custom_cols=False,output_file="tmp/2.csv"):
    # combine acc_version, hardware into one col
    df["acc_version"] = df["acc_version"].astype(str)
    df["hardware"] = df["hardware"].astype(str)
    df["acc_version_hardware"] = df["acc_version"] + "_" + df["hardware"]
    # get unique acc_version_hardware
    acc_version_hardware = df["acc_version_hardware"].unique()
    # get unique model
    model = df["model"].unique()

    # create new df with model thread and column for each acc_version_hardware
    df2 = pd.DataFrame(columns=["model", "thread"] + list(acc_version_hardware))

    # for each model and thread, find the total_latency for each acc_version_hardware
    for m in model:
        for t in [1, 2]:
            # get the rows with the same model and thread
            df3 = df.loc[(df["model"] == m) & (df["thread"] == t)]
            # create a row for the model and thread
            df2 = pd.concat(
                [df2, pd.DataFrame([[m, t]], columns=["model", "thread"])],
                ignore_index=True,
            )
            # get the total_latency for each acc_version_hardware
            for a in acc_version_hardware:
                # get the rows with the same acc_version_hardware
                df4 = df3.loc[df3["acc_version_hardware"] == a]
                # get the sum of the total_latency
                total_latency = df4["total_latency"].sum()
                # add the total_latency to the df2 acc_version_hardware column
                df2.loc[(df2["model"] == m) & (df2["thread"] == t), a] = total_latency

    # sort by CPU
    df2 = df2.sort_values(by=["1_0_CPU", "model", "thread"])
    if custom_cols:
        df2 = custom_cols(df2)
    # #save to csv
    df2.to_csv(output_file, index=False)
    return df2

pd.set_option("display.precision", 0)



In [7]:
# create new df which combines the same model, with new col for each thread's total_latency
df = load_csv("../results/latest.csv")
# merge_rows(df, custom_tconv_cols, "tmp/2.csv")
# merge_rows(df, custom_tconv_cols, "tmp/2.csv")
merge_rows(df, False, "tmp/2.csv")


KeyError: '1_0_CPU'

In [2]:
# load csv
# df = load_csv("../results/latest.csv")
# df = load_csv("../results/mm2imv2.csv")
# latest = load_csv("../results/latest.csv")

# tconv_2 = load_csv("../saved_results/tconv_2.csv")
# mm2imv2_1 = load_csv("../saved_results/mm2imv2_1_post_threading.csv")
# df3 = combine(mm2imv2_1, tconv_2)
# df3.to_csv("tmp/1.csv", index=False)
# merge_rows(df3)




tconv_2 = load_csv("../saved_results/tconv_2.csv")
mm2imv2_1 = load_csv("../saved_results/mm2imv2_1_post_threading_opt2.csv")
df3 = combine(mm2imv2_1, tconv_2)
df3.to_csv("tmp/1.csv", index=False)
# merge_rows(df3)
merge_rows(df3, custom_tconv_cols, "tmp/2.csv")




# create new df which combines the same model, with new col for each thread's total_latency
# df = load_csv("../results/latest.csv")
# merge_rows(df)

/tmp/ipykernel_882940/2722293509.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, i] = df.iloc[:, i].astype(int)


,model,thread,2_4_MM2IM,1_0_CPU,1_0_MM2IM,2_0_MM2IM,stride_x,stride_y,filters,ks,ih,iw,ic,MACs,Outputs,Compute Intensity
13,tconv_1_1_16_3_7_7_32,2,276,4758,424,463,1,1,16,3,7,7,32,225792,784,288
229,tconv_2_2_16_3_7_7_32,2,433,5160,617,633,2,2,16,3,7,7,32,225792,3136,72
21,tconv_1_1_16_3_9_9_32,2,352,7108,573,610,1,1,16,3,9,9,32,373248,1296,288
237,tconv_2_2_16_3_9_9_32,2,570,7418,825,905,2,2,16,3,9,9,32,373248,5184,72
15,tconv_1_1_16_3_7_7_64,2,314,7923,487,645,1,1,16,3,7,7,64,451584,784,576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,tconv_2_2_64_5_11_11_256,1,8191,1362121,10457,22479,2,2,64,5,11,11,256,49561600,30976,1600
210,tconv_1_1_64_7_9_9_256,1,7474,1763687,9777,21025,1,1,64,7,9,9,256,65028096,5184,12544
426,tconv_2_2_64_7_9_9_256,1,9066,1764363,11899,25776,2,2,64,7,9,9,256,65028096,20736,3136
410,tconv_2_2_64_7_11_11_256,1,11826,2655798,15461,37333,2,2,64,7,11,11,256,97140736,30976,3136
